In [13]:
import requests
import time
import pymysql
import os
import json

from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import FOAF, XSD
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint

In [3]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

### Getting list of repositories and authors from the file

In [4]:
cursor.execute("SHOW COLUMNS FROM krw.paper")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    print(r)

('id', 'bigint(20)', 'NO', 'PRI', None, 'auto_increment')
('arxiv_id', 'varchar(255)', 'YES', '', None, '')
('resource_id', 'varchar(255)', 'YES', 'UNI', None, '')
('paper_url', 'varchar(1024)', 'YES', 'MUL', None, '')
('makg_sameAs', 'varchar(1024)', 'YES', 'MUL', 'unprocessed', '')
('authorcount', 'int(11)', 'YES', '', '-1', '')
('title', 'varchar(500)', 'YES', '', None, '')
('abstract', 'varchar(16000)', 'YES', '', None, '')
('url_abs', 'varchar(255)', 'YES', '', None, '')
('url_pdf', 'varchar(255)', 'YES', '', None, '')
('date', 'datetime', 'YES', '', None, '')
('proceeding', 'varchar(255)', 'YES', '', None, '')
('makg_doi', 'varchar(255)', 'YES', '', None, '')
('makg_language', 'varchar(255)', 'YES', '', None, '')
('makg_startpage', 'int(11)', 'YES', '', '-1', '')
('makg_endpage', 'int(11)', 'YES', '', '-1', '')
('makg_citationcount', 'int(11)', 'YES', '', '-1', '')
('makg_referencecount', 'int(11)', 'YES', '', '-1', '')
('makg_publicationdate', 'date', 'YES', '', None, '')
('makg

In [5]:
data = []
paper_ID = {}
cursor.execute("SELECT krw.repo.paper_url, repo_url, author, paper_title, irepoID \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
    
author_dict = {}
author_complex = {}
git_dict = {}
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
        author_complex[d[1]]=[d[2].lower().split()]
        git_dict[d[1]]=d[3]
        paper_ID[d[1]]=d[4] 
    else:
        author_dict[d[1]].append(d[2])
        author_complex[d[1]].append(d[2].lower().split())
print(len(author_dict.keys()))

2079


In [6]:
filename = 'scraped_data.txt'

In [7]:
paper_data = {}
counter = 0
with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            if current_repo in git_dict:
                print(f'Authors: {author_dict[current_repo]}')
                paper_data[git_dict[current_repo]] = {}
                for cont in contributors:
                    author_name = cont[1].replace("_"," ")
                    found_name = author_name in author_dict[current_repo]
                    # Advanced name matching - looking for two lowercase names in lowercased user name
                    if not found_name:
                        for split_name in author_complex[current_repo]:
                            count=0
                            lower = author_name.lower()
                            for name_part in split_name:
                                if name_part in author_name:
                                    count+=1
                            if count>1:
                                counter+=1
                                print(f"DEEPFIND: {split_name} in {author_name}")
                                found_name = True
                                break
                    print(f"{author_name} in {found_name}")
                    if found_name:
                        paper_data[git_dict[current_repo]][author_name] = cont[2:]
                # after saving previous authors, load new repo
            if len(line):
                current_repo = line[0]
                contributors = [line[1:]]  

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

Doing https://github.com/statsu1990/ReZero-Cifar100 with 1 contributors
Doing https://github.com/tair-ai/caffe with 15 contributors
Doing https://github.com/timgaripov/TensorNet-TF with 3 contributors
Doing https://github.com/wq2012/HMRF-EM-image with 1 contributors
Doing https://github.com/JonasRSV/PPO with 1 contributors
Authors: ['John Schulman', 'Filip Wolski', 'Prafulla Dhariwal', 'Alec Radford', 'Oleg Klimov']
Jonas Sunde Valfridsson in False
Doing https://github.com/Ashafix/NeuroMario with 1 contributors
Doing https://github.com/snhwang/p2-continuous-control-SNH with 1 contributors
Doing https://github.com/pomonam/AttentionCluster with 1 contributors
Authors: ['Ashish Vaswani', 'Noam Shazeer', 'Niki Parmar', 'Jakob Uszkoreit', 'Llion Jones', 'Aidan N. Gomez', 'Lukasz Kaiser', 'Illia Polosukhin']
Juhan Bae in False
Doing https://github.com/amdegroot/ssd.pytorch with 9 contributors
Doing https://github.com/black0017/MedicalZooPytorch with 2 contributors
Doing https://github.com/am

Doing https://github.com/majingCUHK/Rumor_RvNN with 3 contributors
Doing https://github.com/esslushy/AbstractArtGenerator with 1 contributors
Doing https://github.com/cersar/3D_detection with 1 contributors
Doing https://github.com/MIC-DKFZ/RegRCNN with 1 contributors
Doing https://github.com/xywei/threshseg with 1 contributors
Doing https://github.com/mahyarnajibi/SSH with 2 contributors
Doing https://github.com/google/ffn with 5 contributors
Doing https://github.com/jiny2001/dcscn-super-resolution with 4 contributors
Doing https://github.com/tdozat/Parser with 1 contributors
Doing https://github.com/sniklaus/pytorch-sepconv with 1 contributors
Doing https://github.com/uclmr/jack with 15 contributors
Doing https://github.com/erikwijmans/Pointnet2_PyTorch with 11 contributors
Doing https://github.com/zomux/neuralcompressor with 3 contributors
Doing https://github.com/VinceMarron/style_transfer with 1 contributors
Doing https://github.com/sour4bh/cifar-10 with 1 contributors
Doing https

### Loading IDs from JSON

In [14]:
ns = Namespace("http://www.scoaring.com/ontology#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
dc = Namespace("http://purl.org/dc/terms/")

In [9]:
# change encoding of file
'''
f = open("datav4.ttl", "r", encoding='latin-1')
output = f.read()
f = open("data.ttl", "a")
f.write(output)
f.close()
'''
# Counting quotation marks to fix naming issues
'''
filepath = 'data.ttl'
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while line:
        line = fp.readline()
        c = line.count('"')
        if c>2:
            print(cnt)
        cnt += 1
'''

'\nfilepath = \'data.ttl\'\nwith open(filepath) as fp:\n    line = fp.readline()\n    cnt = 1\n    while line:\n        line = fp.readline()\n        c = line.count(\'"\')\n        if c>2:\n            print(cnt)\n        cnt += 1\n'

In [10]:
g = Graph()
g.parse("data1.ttl", format="ttl")

<Graph identifier=Nbe678cd7c8524f26a12296655869c1e2 (<class 'rdflib.graph.Graph'>)>

In [ ]:
for s, p, o in g.triples((None, None, None)):
    print(s, o)

In [ ]:
print(g)

In [18]:
print(FOAF)

http://xmlns.com/foaf/0.1/


In [15]:
paper_id = {}

for s, p, o in g.triples((None, dc.title, None)):
    paper_id[str(o)]=s
print(len(paper_id))

2850


In [20]:
authors = {}
for s, p, o in g.triples((None, RDF.type, ns.Author)):
    for s1, p1, o1 in g.triples((s, FOAF.name, None)):
        #print(s, o1)
        authors[str(o1)]=s
print(len(authors))

7267


### Creating graph

In [21]:
software_graph = Graph()
software_graph.namespace_manager.bind('prefix', ns)
missing_papers = []
for paper in paper_data.keys():
    if paper in paper_id:
        print(paper, paper_id[paper])
        for author in paper_data[paper].keys():
            if author in authors:
                print(authors[author].split("#")[-1])
                contr_id = URIRef(f"{paper_id[paper]}_{authors[author].split('#')[-1]}_software")
                print(authors[author], contr_id)
                software_graph.add((contr_id, RDF.type, ns.Contribution))
                software_graph.add((contr_id, RDF.type, owl.NamedIndividual))
                software_graph.add((contr_id, ns.isContributionByAuthor, authors[author]))
                software_graph.add((contr_id, ns.isContributionToPaper, paper_id[paper]))
                software_graph.add((contr_id, ns.role, Literal("software")))
                software_graph.add((contr_id, ns.numberOfCommits, Literal(paper_data[paper][author][0], datatype=XSD.integer)))
                software_graph.add((contr_id, ns.numberOfLinesAddedToRepo, Literal(paper_data[paper][author][1], datatype=XSD.integer)))
                software_graph.add((contr_id, ns.numberOfLinesDeleted, Literal(paper_data[paper][author][2], datatype=XSD.integer)))
    else:
        missing_papers.append(paper)
        print('Not in the list')

Unsupervised Training for 3D Morphable Model Regression http://www.scoaring.com/ontology#P1220582
A79513
http://www.scoaring.com/ontology#A79513 http://www.scoaring.com/ontology#P1220582_A79513_software
Adaptive Affinity Fields for Semantic Segmentation http://www.scoaring.com/ontology#P1179582
A143384
http://www.scoaring.com/ontology#A143384 http://www.scoaring.com/ontology#P1179582_A143384_software
A73462
http://www.scoaring.com/ontology#A73462 http://www.scoaring.com/ontology#P1179582_A73462_software
The Double Sphere Camera Model http://www.scoaring.com/ontology#P1213491
A147169
http://www.scoaring.com/ontology#A147169 http://www.scoaring.com/ontology#P1213491_A147169_software
A104620
http://www.scoaring.com/ontology#A104620 http://www.scoaring.com/ontology#P1213491_A104620_software
Banach Wasserstein GAN http://www.scoaring.com/ontology#P1176316
Convolutional Neural Networks for Sentence Classification http://www.scoaring.com/ontology#P1283847
Bottom-Up and Top-Down Attention for 

In [22]:
software_graph.serialize(destination='software_contributions.ttl', format='ttl')

In [24]:
g2 = Graph()
g2.parse("v4_role_email.ttl", format="ttl")

<Graph identifier=N06cf56c73f524cb0a572445266f3c8dc (<class 'rdflib.graph.Graph'>)>

In [25]:
final_graph = g + software_graph + g2

In [26]:
final_graph.serialize(destination='full_graph.ttl', format='ttl')